In [40]:
# Importing libraries
import pandas as pd
import warnings
import time

start = time.time()

# Set warnings off
warnings.filterwarnings("ignore")

Data Cleaning

In [41]:
df = pd.read_csv('Data_FeatureFinish.csv')

In [42]:
df.head()

,Fecha,Hora,Clase de Vuelo,Clasificación Vuelo,Tipo Movimiento,Aeropuerto,Origen/Destino,Aerolinea Nombre,Aeronave,Pasajeros,PAX,Calidad dato
0,1/1/2019,00:01:00,REGULAR,Doméstico,Aterrizaje,EZE,SAL,AEROLINEAS ARGENTINAS SA,BO-B-737-76N,88,44,DEFINITIVO
1,1/1/2019,00:01:00,REGULAR,Internacional,Aterrizaje,EZE,SBGL,TRANSPORTES AEREOS DEL MERCOSUR,NaN,165,165,DEFINITIVO
2,1/1/2019,00:03:00,REGULAR,Doméstico,Aterrizaje,AER,SIS,AUSTRAL LINEAS AEREAS-CIELOS DEL SUR S.A,EMB-ERJ190100IGW,22,11,DEFINITIVO
3,1/1/2019,00:04:00,REGULAR,Internacional,Aterrizaje,EZE,SBGR,TRANSPORTES AEREOS DEL MERCOSUR,NaN,69,69,DEFINITIVO
4,1/1/2019,00:06:00,REGULAR,Internacional,Aterrizaje,AER,SBGR,LAN ARGENTINA S.A. (LATAM AIRLINES),AIB-A-320-233,53,53,DEFINITIVO


In [43]:
df.columns

Index(['Fecha', 'Hora', 'Clase de Vuelo', 'Clasificación Vuelo',
       'Tipo Movimiento', 'Aeropuerto', 'Origen/Destino', 'Aerolinea Nombre',
       'Aeronave', 'Pasajeros', 'PAX', 'Calidad dato'],
      dtype='object')

In [44]:
# Columns to analyze
columns = ['Clase de Vuelo', 'Clasificación Vuelo', 'Tipo Movimiento', 'Calidad dato']

# Checking the unique values of each column
for column in columns:
    print(f'Unique values for the column: {column}')
    print(df[column].unique())

    print('\n************************************************************************\n')

Unique values for the column: Clase de Vuelo
['REGULAR' 'VUELOS PRIVADOS NACIONALES' 'VUELOS OFICIALES NACIONALES'
 'NO REGULAR' 'VUELOS PRIVADO CON MATRICULA EXTRANJERA' 'TRABAJO AEREO'
 'VUELOS DE ADIESTRAMIENTO' 'VUELOS ESCUELA'
 'VUELOS OFICIALES EXTRANJEROS' 'ESCUELA (NO VIGENTE)' 'Regular'
 'Vuelo Privado con Matrícula Nacional' 'No Regular'
 'Vuelo Privado con Matrícula Extranjera' 'Trabajo Aéreo'
 'Vuelo de Adiestramiento' 'Vuelo Oficial Extranjero'
 'Vuelo Oficial Nacional' 'Vuelo Escuela']

************************************************************************

Unique values for the column: Clasificación Vuelo
['Doméstico' 'Internacional' 'Dom' 'Inter']

************************************************************************

Unique values for the column: Tipo Movimiento
['Aterrizaje' 'Despegue' nan]

************************************************************************

Unique values for the column: Calidad dato
['DEFINITIVO' 'PROVISORIO']

****************************

In [45]:
# Recoding the values of column 'Clase de Vuelo'
df['Clase de Vuelo'] = df['Clase de Vuelo'].str.capitalize()

In [46]:
df['Clase de Vuelo'].unique()

array(['Regular', 'Vuelos privados nacionales',
       'Vuelos oficiales nacionales', 'No regular',
       'Vuelos privado con matricula extranjera', 'Trabajo aereo',
       'Vuelos de adiestramiento', 'Vuelos escuela',
       'Vuelos oficiales extranjeros', 'Escuela (no vigente)',
       'Vuelo privado con matrícula nacional',
       'Vuelo privado con matrícula extranjera', 'Trabajo aéreo',
       'Vuelo de adiestramiento', 'Vuelo oficial extranjero',
       'Vuelo oficial nacional', 'Vuelo escuela'], dtype=object)

In [47]:
# Define how the values are going to change
fligth_class = {
    'Vuelos privados nacionales': 'Vuelo privado nacional',
    'Vuelos oficiales nacionales': 'Vuelo oficial nacional',
    'Vuelos privado con matricula extranjera': 'Vuelo privado con matrícula extranjera',
    'Trabajo aereo': 'Trabajo aéreo',
    'Vuelos de adiestramiento': 'Vuelo de adiestramiento',
    'Vuelos escuela': 'Vuelo escuela',
    'Vuelos oficiales extranjeros': 'Vuelo oficial extranjero'
}

# Replacing the values
df['Clase de Vuelo'].replace(fligth_class, inplace=True)

In [48]:
# Replacing the values of column 'Clasificación Vuelo'
df['Clasificación Vuelo'] = df['Clasificación Vuelo'].replace({
    'Dom': 'Doméstico',
    'Inter': 'Internacional'
})

In [49]:
# Checking the unique values of each column
for column in columns:
    print(f'Unique values for the column: {column}')
    print(df[column].unique())

    print('\n************************************************************************\n')

Unique values for the column: Clase de Vuelo
['Regular' 'Vuelo privado nacional' 'Vuelo oficial nacional' 'No regular'
 'Vuelo privado con matrícula extranjera' 'Trabajo aéreo'
 'Vuelo de adiestramiento' 'Vuelo escuela' 'Vuelo oficial extranjero'
 'Escuela (no vigente)' 'Vuelo privado con matrícula nacional']

************************************************************************

Unique values for the column: Clasificación Vuelo
['Doméstico' 'Internacional']

************************************************************************

Unique values for the column: Tipo Movimiento
['Aterrizaje' 'Despegue' nan]

************************************************************************

Unique values for the column: Calidad dato
['DEFINITIVO' 'PROVISORIO']

************************************************************************



In [50]:
data_columns = []

for column in columns:
    data_column = df.groupby(by=column).size().reset_index().sort_values(by=0, ascending=False).rename(columns={0:'Quantity'})
    data_columns.append(data_column)

In [51]:
data_columns[0]

,Clase de Vuelo,Quantity
2,Regular,1162018
5,Vuelo escuela,289055
9,Vuelo privado con matrícula nacional,272590
1,No regular,173950
7,Vuelo oficial nacional,139675
10,Vuelo privado nacional,72277
4,Vuelo de adiestramiento,19335
8,Vuelo privado con matrícula extranjera,16722
3,Trabajo aéreo,8724
6,Vuelo oficial extranjero,1526


In [52]:
# Delete the register with the value 'Escuela (no vigente)'
df = df[df['Clase de Vuelo'] != 'Escuela (no vigente)']

In [53]:
data_columns = []

for column in columns:
    data_column = df.groupby(by=column).size().reset_index().sort_values(by=0, ascending=False).rename(columns={0:'Quantity'})
    data_columns.append(data_column)

In [54]:
# Checking the frequency of each value from the column 'Clase de Vuelo'
data_columns[0]

,Clase de Vuelo,Quantity
1,Regular,1162018
4,Vuelo escuela,289055
8,Vuelo privado con matrícula nacional,272590
0,No regular,173950
6,Vuelo oficial nacional,139675
9,Vuelo privado nacional,72277
3,Vuelo de adiestramiento,19335
7,Vuelo privado con matrícula extranjera,16722
2,Trabajo aéreo,8724
5,Vuelo oficial extranjero,1526


In [55]:
# Checking the values for the column 'Clasificación Vuelo'
data_columns[1]

,Clasificación Vuelo,Quantity
0,Doméstico,1799863
1,Internacional,356009


In [56]:
# Checking the values for the column 'Tipo Movimiento'
data_columns[2]

,Tipo Movimiento,Quantity
1,Despegue,1086137
0,Aterrizaje,1069732


In [57]:
# Checking the values for the column 'Calidad dato'
data_columns[3]

,Calidad dato,Quantity
0,DEFINITIVO,2075202
1,PROVISORIO,80670


Fixing NaN Values

In [58]:
# Checking the NaN values
df.isna().sum()

Fecha                       0
Hora                        0
Clase de Vuelo              0
Clasificación Vuelo         0
Tipo Movimiento             3
Aeropuerto                  0
Origen/Destino              3
Aerolinea Nombre       182406
Aeronave                91661
Pasajeros                   0
PAX                         0
Calidad dato                0
dtype: int64

In [59]:
# Delete NaN Values from 'Tipo Movimiento' and 'Origen/Destino' which are the same registers.
df = df[~df['Tipo Movimiento'].isna()]

In [60]:
# Checking the NaN values
df.isna().sum()

Fecha                       0
Hora                        0
Clase de Vuelo              0
Clasificación Vuelo         0
Tipo Movimiento             0
Aeropuerto                  0
Origen/Destino              0
Aerolinea Nombre       182404
Aeronave                91661
Pasajeros                   0
PAX                         0
Calidad dato                0
dtype: int64

In [61]:
# To fix the NaN Values of column 'Aerolinea Nombre' I am going to catch the 'Aerolinea Name' repeated the most for each 'Aeronave' value, and then fill the missing values on the column 'Aerolinea Nombre' with those.

# Catching the 'Aeronave' values only where 'Aerolinea Nombre' and 'Aeronave' have real values
plane_name = set(df[(~df['Aerolinea Nombre'].isna()) & (df['Aerolinea Nombre'] != '0') & (~df['Aeronave'].isna()) & (df['Aeronave'] != '0')]['Aeronave'])
plane_name.remove(0)

df_planes = df[(~df['Aerolinea Nombre'].isna()) & (df['Aerolinea Nombre'] != '0')]

setup_planes = {}
for plane in plane_name:
    data_reduce = df_planes[df_planes['Aeronave'] == plane].groupby(by='Aerolinea Nombre').size().reset_index().rename(columns={0:'Quantity'})
    max_value = data_reduce['Quantity'].idxmax()
    max_value = data_reduce.loc[max_value, 'Aerolinea Nombre']
    setup_planes[plane] = max_value

print(setup_planes)

{'TEA-P2008': 'AEROTEC ARGENTINA S.A.', 'PAA-PA-A-28R201T': 'ECHO VICTOR', 'PA-PA-28-181': 'HANGAR UNO S.A', 'TEA-P-MENTOR': 'AEROTEC ARGENTINA S.A.', 'BEL-429': 'AEROLINEAS ARGENTINAS SA', 'BO-B-767-3Q8': 'LAN ARGENTINA S.A. (LATAM AIRLINES)', 'ARH-MBB-BK117 C2': 'PRESIDENCIA DE LA NACION', 'SI-SIPA 1000': 'AEROLINEAS ARGENTINAS SA', 'EUR-MBB-BK-117C2': 'ESC.FEDERAL DE AVIACION POLICIAL', 'AIB-A-320-212': 'LAN ARGENTINA S.A. (LATAM AIRLINES)', 'CH-BD-100-1A10': 'AMBITO AGROPECUARIO S.A.', 'RW-DARTER-100': 'VOLAR', 'PA-PA-22': 'AERO CLUB EL BOLSON', 'RAY-HAWKER 800XP': 'AERONAUTICA S.A.', 'RO-R-44': 'HANGAR UNO S.A', 'HU-269-C': 'AEROLINEAS ARGENTINAS SA', 'CE-C-172-M': 'RUFU', 'CE-A-182-L': 'ESC.FEDERAL DE AVIACION POLICIAL', 'BO-767-316ER': 'RUFU', 'AIR-AT-402-B': 'JETSMART AIRLINES S.A.', 'BE-B-300': 'SUNDOWN JET S.A.', 'CE-310': 'AEROLINEAS ARGENTINAS SA', 'BO-737-86J': 'FB LÍNEAS AÉREAS - FLYBONDI', 'ST-V-77': 'AEROLINEAS ARGENTINAS SA', 'AIB-A320-214': 'AVIAN LINEAS AEREAS S.A. (

In [62]:
# Fillig the missing values on 'Aerolinea Nombre' with the data from 'setup_planes' variables.
for plane, airline in setup_planes.items():
    df.loc[df['Aeronave'] == plane, 'Aerolinea Nombre'] = airline

In [63]:
# Checking the NaN values
df.isna().sum()

Fecha                      0
Hora                       0
Clase de Vuelo             0
Clasificación Vuelo        0
Tipo Movimiento            0
Aeropuerto                 0
Origen/Destino             0
Aerolinea Nombre       29781
Aeronave               91661
Pasajeros                  0
PAX                        0
Calidad dato               0
dtype: int64

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2155869 entries, 0 to 2155872
Data columns (total 12 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   Fecha                object
 1   Hora                 object
 2   Clase de Vuelo       object
 3   Clasificación Vuelo  object
 4   Tipo Movimiento      object
 5   Aeropuerto           object
 6   Origen/Destino       object
 7   Aerolinea Nombre     object
 8   Aeronave             object
 9   Pasajeros            int64 
 10  PAX                  object
 11  Calidad dato         object
dtypes: int64(1), object(11)
memory usage: 213.8+ MB


In [65]:
df.dropna(inplace=True)

In [66]:
# Checking the NaN values
df.isna().sum()

Fecha                  0
Hora                   0
Clase de Vuelo         0
Clasificación Vuelo    0
Tipo Movimiento        0
Aeropuerto             0
Origen/Destino         0
Aerolinea Nombre       0
Aeronave               0
Pasajeros              0
PAX                    0
Calidad dato           0
dtype: int64

In [67]:
del(df['Aeronave'])
del(df['Calidad dato'])

In [68]:
df.head()

,Fecha,Hora,Clase de Vuelo,Clasificación Vuelo,Tipo Movimiento,Aeropuerto,Origen/Destino,Aerolinea Nombre,Pasajeros,PAX
0,1/1/2019,00:01:00,Regular,Doméstico,Aterrizaje,EZE,SAL,AEROLINEAS ARGENTINAS SA,88,44
2,1/1/2019,00:03:00,Regular,Doméstico,Aterrizaje,AER,SIS,AEROLINEAS ARGENTINAS SA,22,11
4,1/1/2019,00:06:00,Regular,Internacional,Aterrizaje,AER,SBGR,LAN ARGENTINA S.A. (LATAM AIRLINES),53,53
5,1/1/2019,00:09:00,Regular,Doméstico,Despegue,TUC,AER,LAN ARGENTINA S.A. (LATAM AIRLINES),156,78
6,1/1/2019,00:11:00,Regular,Doméstico,Despegue,MDP,AER,AEROLINEAS ARGENTINAS SA,4,2


In [69]:
df.to_csv('Data_finish.csv')